# Status #

- [ ] Outline
- [ ] Introduction
- [ ] Exercise 1
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 2
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 3
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 4
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Conclusion

# Introduction #

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex4 import *

# Moving Windows #

### 1) Explore Window Sizes

In [ ]:

q_1.check()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

### 2) Explore Stride


In [ ]:

q_2.check()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

# Conclusion #


In fact, the moving windows we looked at in this tutorial are also widely used in models for time series and natural language.